In [1]:
import pandas as pd
import geopandas as gpd
import os
import json
import numpy as np

In [2]:
DATA_DIR = '../local/g-naf_feb24_allstates_gda2020_psv_1014/G-NAF/G-NAF FEBRUARY 2024'
PSV_FILES_STANDARD = [(filename, os.path.join(DATA_DIR, 'Standard', filename)) for filename in os.listdir(os.path.join(DATA_DIR, 'Standard'))]
PSV_FILES_AUTHORITY_CODE = [(filename, os.path.join(DATA_DIR, 'Authority Code', filename)) for filename in os.listdir(os.path.join(DATA_DIR, 'Authority Code'))]
PSV_FILES = PSV_FILES_STANDARD + PSV_FILES_AUTHORITY_CODE
assert all(filename.endswith('_psv.psv') for filename, filepath in PSV_FILES)
PSV_FILES = [(filename.removesuffix('_psv.psv'), filepath) for filename, filepath in PSV_FILES]

In [3]:
GNAF_DFK : dict[str, pd.DataFrame] = {}
for filename, filepath in PSV_FILES:
    if filename.startswith('VIC_') or filename.startswith('Authority_Code_'):
        GNAF_DFK[filename] = pd.read_csv(filepath, sep='|', dtype=str) 
        print(f'{filename}: {GNAF_DFK[filename].shape}')
# 6m 30s - 8m 30s for VIC only

VIC_ADDRESS_ALIAS: (215962, 7)
VIC_ADDRESS_DEFAULT_GEOCODE: (4241360, 7)
VIC_ADDRESS_DETAIL: (4241360, 35)
VIC_ADDRESS_FEATURE: (35336, 6)
VIC_ADDRESS_MESH_BLOCK_2016: (4241360, 6)
VIC_ADDRESS_MESH_BLOCK_2021: (4241360, 6)
VIC_ADDRESS_SITE_GEOCODE: (8087748, 13)
VIC_ADDRESS_SITE: (4241360, 5)
VIC_LOCALITY_ALIAS: (3686, 8)
VIC_LOCALITY_NEIGHBOUR: (17236, 5)
VIC_LOCALITY_POINT: (2996, 7)
VIC_LOCALITY: (2996, 9)
VIC_MB_2016: (85014, 4)
VIC_MB_2021: (88739, 4)
VIC_PRIMARY_SECONDARY: (1225385, 7)
VIC_STATE: (1, 5)
VIC_STREET_LOCALITY_ALIAS: (11678, 8)
VIC_STREET_LOCALITY_POINT: (195051, 8)
VIC_STREET_LOCALITY: (202488, 11)
Authority_Code_ADDRESS_ALIAS_TYPE_AUT: (8, 3)
Authority_Code_ADDRESS_CHANGE_TYPE_AUT: (511, 3)
Authority_Code_ADDRESS_TYPE_AUT: (3, 3)
Authority_Code_FLAT_TYPE_AUT: (54, 3)
Authority_Code_GEOCODED_LEVEL_TYPE_AUT: (8, 3)
Authority_Code_GEOCODE_RELIABILITY_AUT: (6, 3)
Authority_Code_GEOCODE_TYPE_AUT: (29, 3)
Authority_Code_LEVEL_TYPE_AUT: (16, 3)
Authority_Code_LOCALITY_ALI

In [6]:
GNAF_DFK['VIC_MB_2021']['MB_2021_PID'].is_unique

True

In [9]:
GNAF_DFK['VIC_STREET_LOCALITY'].head()

,STREET_LOCALITY_PID,DATE_CREATED,DATE_RETIRED,STREET_CLASS_CODE,STREET_NAME,STREET_TYPE_CODE,STREET_SUFFIX_CODE,LOCALITY_PID,GNAF_STREET_PID,GNAF_STREET_CONFIDENCE,GNAF_RELIABILITY_CODE
0,VIC1962328,2021-07-06,NaN,C,CLARENCE,STREET,NaN,loc201e214973bd,253028435,2,4
1,VIC2031591,2021-07-06,NaN,C,STABLE,COURT,NaN,locb10305db77b5,253059724,2,4
2,VIC2049613,2021-07-06,NaN,C,TUDOR,COURT,NaN,loc00d1503504f1,253062827,2,4
3,VIC1967443,2021-07-06,NaN,C,DEIDRE,STREET,NaN,locd755ccb7197e,253031330,2,4
4,VIC1968231,2021-07-06,NaN,C,GEORGE,STREET,NaN,loca2dba6250aa2,253036166,2,4


In [12]:
GNAF_DFK['VIC_STREET_LOCALITY']['STREET_CLASS_CODE'].value_counts()

STREET_CLASS_CODE
C    180575
U     21913
Name: count, dtype: int64

In [41]:
east_streets_df = GNAF_DFK['VIC_STREET_LOCALITY'].query('(STREET_NAME == "EAST") & (STREET_TYPE_CODE == "STREET")')
east_streets_df = pd.merge(east_streets_df, GNAF_DFK['VIC_LOCALITY'], left_on='LOCALITY_PID', right_on='LOCALITY_PID', suffixes=('_street', '_locality'), how='left')
east_streets_df = pd.merge(east_streets_df, GNAF_DFK['VIC_ADDRESS_DETAIL'], left_on='STREET_LOCALITY_PID', right_on='STREET_LOCALITY_PID', suffixes=('_sl', '_address'), how='left')

['STREET_LOCALITY_PID', 'DATE_CREATED_street', 'DATE_RETIRED_street',
       'STREET_CLASS_CODE', 'STREET_NAME', 'STREET_TYPE_CODE',
       'STREET_SUFFIX_CODE', 'LOCALITY_PID_sl', 'GNAF_STREET_PID',
       'GNAF_STREET_CONFIDENCE', 'GNAF_RELIABILITY_CODE_street',
       'DATE_CREATED_locality', 'DATE_RETIRED_locality', 'LOCALITY_NAME',
       'PRIMARY_POSTCODE', 'LOCALITY_CLASS_CODE', 'STATE_PID',
       'GNAF_LOCALITY_PID', 'GNAF_RELIABILITY_CODE_locality',
       'ADDRESS_DETAIL_PID', 'DATE_CREATED', 'DATE_LAST_MODIFIED',
       'DATE_RETIRED', 'BUILDING_NAME', 'LOT_NUMBER_PREFIX', 'LOT_NUMBER',
       'LOT_NUMBER_SUFFIX', 'FLAT_TYPE_CODE', 'FLAT_NUMBER_PREFIX',
       'FLAT_NUMBER', 'FLAT_NUMBER_SUFFIX', 'LEVEL_TYPE_CODE',
       'LEVEL_NUMBER_PREFIX', 'LEVEL_NUMBER', 'LEVEL_NUMBER_SUFFIX',
       'NUMBER_FIRST_PREFIX', 'NUMBER_FIRST', 'NUMBER_FIRST_SUFFIX',
       'NUMBER_LAST_PREFIX', 'NUMBER_LAST', 'NUMBER_LAST_SUFFIX',
       'LOCATION_DESCRIPTION', 'LOCALITY_PID_address', 'ALIAS_PRINCIPAL',
       'POSTCODE', 'PRIVATE_STREET', 'LEGAL_PARCEL_ID', 'CONFIDENCE',
       'ADDRESS_SITE_PID', 'LEVEL_GEOCODED_CODE', 'PROPERTY_PID',
       'GNAF_PROPERTY_PID', 'PRIMARY_SECONDARY']

east_streets_ascot_vale_df = east_streets_df.query('LOCALITY_NAME == "ASCOT VALE"')
# [['ADDRESS_DETAIL_ID', 'STREET_NAME', 'STREET_TYPE_CODE', 'LOCALITY_NAME', 'POSTCODE', 'BUILDING_NAME', 'LOT_NUMBER_PREFIX', 'LOT_NUMBER', 'LOT_NUMBER_SUFFIX', 'FLAT_TYPE_CODE', 'FLAT_NUMBER_PREFIX', 'FLAT_NUMBER', 'FLAT_NUMBER_SUFFIX', 'LEVEL_TYPE_CODE', 'LEVEL_NUMBER_PREFIX', 'LEVEL_NUMBER', 'LEVEL_NUMBER_SUFFIX', 'NUMBER_FIRST_PREFIX', 'NUMBER_FIRST', 'NUMBER_FIRST_SUFFIX', 'NUMBER_LAST_PREFIX', 'NUMBER_LAST', 'NUMBER_LAST_SUFFIX']]

In [54]:
for col in GNAF_DFK['VIC_STREET_LOCALITY'].columns:
    if GNAF_DFK['VIC_STREET_LOCALITY'][col].isna().all():
        # del east_streets_ascot_vale_df[col]
        print(col)

DATE_RETIRED


In [66]:
east_streets_ascot_vale_df[['ADDRESS_DETAIL_PID', 'NUMBER_FIRST', 'NUMBER_FIRST_SUFFIX', 'STREET_NAME', 'STREET_TYPE_CODE', 'LOCALITY_NAME', 'POSTCODE', ]]

,ADDRESS_DETAIL_PID,NUMBER_FIRST,NUMBER_FIRST_SUFFIX,STREET_NAME,STREET_TYPE_CODE,LOCALITY_NAME,POSTCODE
16,GAVIC422031364,3,A,EAST,STREET,ASCOT VALE,3032
17,GAVIC419912825,15,A,EAST,STREET,ASCOT VALE,3032
18,GAVIC419918111,9,NaN,EAST,STREET,ASCOT VALE,3032
19,GAVIC422088925,1,NaN,EAST,STREET,ASCOT VALE,3032
20,GAVIC421119212,15,B,EAST,STREET,ASCOT VALE,3032
21,GAVIC421874736,2,B,EAST,STREET,ASCOT VALE,3032
22,GAVIC419538220,2,NaN,EAST,STREET,ASCOT VALE,3032
23,GAVIC419837620,11,NaN,EAST,STREET,ASCOT VALE,3032
24,GAVIC420495430,15,C,EAST,STREET,ASCOT VALE,3032
25,GAVIC421584717,2,A,EAST,STREET,ASCOT VALE,3032


In [61]:
GNAF_DFK['VIC_ADDRESS_DETAIL'].head()

,ADDRESS_DETAIL_PID,DATE_CREATED,DATE_LAST_MODIFIED,DATE_RETIRED,BUILDING_NAME,LOT_NUMBER_PREFIX,LOT_NUMBER,LOT_NUMBER_SUFFIX,FLAT_TYPE_CODE,FLAT_NUMBER_PREFIX,...,ALIAS_PRINCIPAL,POSTCODE,PRIVATE_STREET,LEGAL_PARCEL_ID,CONFIDENCE,ADDRESS_SITE_PID,LEVEL_GEOCODED_CODE,PROPERTY_PID,GNAF_PROPERTY_PID,PRIMARY_SECONDARY
0,GAVIC423790614,2006-10-25,2021-07-07,NaN,NaN,NaN,NaN,NaN,SHOP,NaN,...,P,3690,NaN,CP166716,1,423881576,7,NaN,100577707,S
1,GAVIC419857545,2004-04-29,2021-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,P,3777,NaN,2\LP72345,2,419994673,7,NaN,1177087,NaN
2,GAVIC421052916,2004-04-29,2021-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,P,3123,NaN,2\SP25732,2,421201621,7,NaN,131243,NaN
3,GAVIC420173645,2004-04-29,2021-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,P,3123,NaN,1\LP37836,2,420302828,7,NaN,131246,NaN
4,GAVIC421052921,2004-04-29,2021-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,P,3123,NaN,1\TP345739,2,421201626,7,NaN,131247,NaN


In [110]:
GNAF_DFK['VIC_ADDRESS_SITE']['ADDRESS_SITE_PID'].is_unique, GNAF_DFK['VIC_ADDRESS_SITE_GEOCODE']['ADDRESS_SITE_PID'].is_unique

(True, False)

In [131]:
GNAF_DFK['VIC_ADDRESS_SITE'].head()
# Find intersection of two sets of columns
set(GNAF_DFK['VIC_LOCALITY'].columns) & set(GNAF_DFK['VIC_ADDRESS_DETAIL'].columns)
set(GNAF_DFK['VIC_STREET_LOCALITY'].columns) & set(GNAF_DFK['VIC_LOCALITY'].columns)
set(GNAF_DFK['VIC_STREET_LOCALITY'].columns) & set(GNAF_DFK['VIC_ADDRESS_DETAIL'].columns)


{'DATE_CREATED', 'DATE_RETIRED', 'LOCALITY_PID', 'STREET_LOCALITY_PID'}

In [123]:
assert GNAF_DFK['VIC_ADDRESS_DETAIL']['STREET_LOCALITY_PID'].notna().all()

True

In [9]:
assert GNAF_DFK['VIC_ADDRESS_DETAIL']['STREET_LOCALITY_PID'].notna().all()
assert GNAF_DFK['VIC_STREET_LOCALITY']['LOCALITY_PID'].notna().all()
df_streets = pd.merge(GNAF_DFK['VIC_STREET_LOCALITY'][['STREET_LOCALITY_PID', 'LOCALITY_PID']], GNAF_DFK['VIC_LOCALITY'][['LOCALITY_PID', 'LOCALITY_NAME']], on='LOCALITY_PID', suffixes=('', '-l'), how='left')
k = GNAF_DFK['VIC_ADDRESS_DETAIL'][['STREET_LOCALITY_PID', 'LOCALITY_PID']].drop_duplicates()
dk = pd.merge(k, df_streets, on='STREET_LOCALITY_PID', suffixes=('', '-s'), how='left')
assert dk['LOCALITY_PID'].equals(dk['LOCALITY_PID-s'])
del df_streets, k, dk

In [13]:
GNAF_DFK['VIC_ADDRESS_DETAIL'].head()

,ADDRESS_DETAIL_PID,DATE_CREATED,DATE_LAST_MODIFIED,DATE_RETIRED,BUILDING_NAME,LOT_NUMBER_PREFIX,LOT_NUMBER,LOT_NUMBER_SUFFIX,FLAT_TYPE_CODE,FLAT_NUMBER_PREFIX,...,ALIAS_PRINCIPAL,POSTCODE,PRIVATE_STREET,LEGAL_PARCEL_ID,CONFIDENCE,ADDRESS_SITE_PID,LEVEL_GEOCODED_CODE,PROPERTY_PID,GNAF_PROPERTY_PID,PRIMARY_SECONDARY
0,GAVIC423790614,2006-10-25,2021-07-07,NaN,NaN,NaN,NaN,NaN,SHOP,NaN,...,P,3690,NaN,CP166716,1,423881576,7,NaN,100577707,S
1,GAVIC419857545,2004-04-29,2021-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,P,3777,NaN,2\LP72345,2,419994673,7,NaN,1177087,NaN
2,GAVIC421052916,2004-04-29,2021-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,P,3123,NaN,2\SP25732,2,421201621,7,NaN,131243,NaN
3,GAVIC420173645,2004-04-29,2021-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,P,3123,NaN,1\LP37836,2,420302828,7,NaN,131246,NaN
4,GAVIC421052921,2004-04-29,2021-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,P,3123,NaN,1\TP345739,2,421201626,7,NaN,131247,NaN


In [18]:
locality_pid = GNAF_DFK['VIC_LOCALITY'].query('LOCALITY_NAME == "CLAYTON"').iloc[0]['LOCALITY_PID']
street_locality_pid = GNAF_DFK['VIC_STREET_LOCALITY'].query(f'LOCALITY_PID == "{locality_pid}" and STREET_NAME == "GLENBROOK"').iloc[0]['STREET_LOCALITY_PID']
address_site_pid = GNAF_DFK['VIC_ADDRESS_DETAIL'].query(f'STREET_LOCALITY_PID == "{street_locality_pid}" and NUMBER_FIRST == "39"').iloc[0]['ADDRESS_SITE_PID']
GNAF_DFK['VIC_ADDRESS_SITE_GEOCODE'].query(f'ADDRESS_SITE_PID == "{address_site_pid}"')


,ADDRESS_SITE_GEOCODE_PID,DATE_CREATED,DATE_RETIRED,ADDRESS_SITE_PID,GEOCODE_SITE_NAME,GEOCODE_SITE_DESCRIPTION,GEOCODE_TYPE_CODE,RELIABILITY_CODE,BOUNDARY_EXTENT,PLANIMETRIC_ACCURACY,ELEVATION,LONGITUDE,LATITUDE
6071817,PC421631940,2018-05-11,NaN,421631940,NaN,NaN,PC,2,NaN,NaN,NaN,145.12341431,-37.90983361
8055038,4056258,2012-11-01,NaN,421631940,NaN,NaN,FCS,2,NaN,NaN,NaN,145.1235659,-37.90984585


In [ ]:
GNAF_DFK['VIC_ADDRESS_DETAIL']['ADDRESS_DETAIL_PID']

In [45]:
GNAF_DFK['VIC_ADDRESS_DETAIL'].query(f'ADDRESS_DETAIL_PID == "GAVIC421038791"')

,ADDRESS_DETAIL_PID,DATE_CREATED,DATE_LAST_MODIFIED,DATE_RETIRED,BUILDING_NAME,LOT_NUMBER_PREFIX,LOT_NUMBER,LOT_NUMBER_SUFFIX,FLAT_TYPE_CODE,FLAT_NUMBER_PREFIX,...,ALIAS_PRINCIPAL,POSTCODE,PRIVATE_STREET,LEGAL_PARCEL_ID,CONFIDENCE,ADDRESS_SITE_PID,LEVEL_GEOCODED_CODE,PROPERTY_PID,GNAF_PROPERTY_PID,PRIMARY_SECONDARY
243720,GAVIC421038791,2006-01-14,2021-08-05,NaN,FLINDERS STREET RAILWAY STATION,NaN,NaN,NaN,NaN,NaN,...,P,3000,NaN,21A~4\PP5514C,0,421180459,7,NaN,204315992,P


In [46]:
GNAF_DFK['VIC_ADDRESS_DETAIL'][GNAF_DFK['VIC_ADDRESS_DETAIL']['BUILDING_NAME'].notna() & GNAF_DFK['VIC_ADDRESS_DETAIL']['BUILDING_NAME'].str.contains('RAILWAY STATION')]

,ADDRESS_DETAIL_PID,DATE_CREATED,DATE_LAST_MODIFIED,DATE_RETIRED,BUILDING_NAME,LOT_NUMBER_PREFIX,LOT_NUMBER,LOT_NUMBER_SUFFIX,FLAT_TYPE_CODE,FLAT_NUMBER_PREFIX,...,ALIAS_PRINCIPAL,POSTCODE,PRIVATE_STREET,LEGAL_PARCEL_ID,CONFIDENCE,ADDRESS_SITE_PID,LEVEL_GEOCODED_CODE,PROPERTY_PID,GNAF_PROPERTY_PID,PRIMARY_SECONDARY
83199,GAVIC719756142,2018-05-07,2021-08-05,NaN,MIDDLE PARK RAILWAY STATION,NaN,NaN,NaN,NaN,NaN,...,P,3182,NaN,RES1\PS329924,0,715141165,7,NaN,173805078,NaN
243720,GAVIC421038791,2006-01-14,2021-08-05,NaN,FLINDERS STREET RAILWAY STATION,NaN,NaN,NaN,NaN,NaN,...,P,3000,NaN,21A~4\PP5514C,0,421180459,7,NaN,204315992,P
314448,GAVIC719205689,2017-10-17,2021-08-05,NaN,WOODEND RAILWAY STATION,NaN,NaN,NaN,NaN,NaN,...,P,3442,NaN,9~42\PP5874,0,714590722,7,NaN,52521450,NaN
340443,GAVIC719613240,2019-10-20,2021-08-05,NaN,LEONGATHA RAILWAY STATION,NaN,NaN,NaN,NaN,NaN,...,P,3953,NaN,38A~22\PP5460,0,714998254,7,NaN,430593430,NaN
419437,GAVIC425469095,2014-07-17,2023-05-03,NaN,NEW MARKET RAILWAY STATION,NaN,NaN,NaN,NaN,NaN,...,P,3031,NaN,1\TP954249,1,425557385,7,NaN,420275804,NaN
737415,GAVIC421603772,2017-10-26,2021-08-05,NaN,NORTH MELBOURNE RAILWAY STATION,NaN,NaN,NaN,NaN,NaN,...,P,3003,NaN,3~73B\PP5514B,1,421777674,7,NaN,203461331,NaN
776519,GAVIC719790566,2018-07-20,2021-08-05,NaN,SEA LAKE RAILWAY STATION,NaN,NaN,NaN,NaN,NaN,...,P,3533,NaN,2003\PP5703,0,715175589,7,NaN,427829583,NaN
855829,GAVIC719535024,2017-10-17,2021-08-05,NaN,ST ALBANS RAILWAY STATION,NaN,NaN,NaN,NaN,NaN,...,P,3021,NaN,1\TP956524,1,714920056,7,NaN,426473580,P
901758,GAVIC425467478,2021-04-09,2021-08-05,NaN,MOONEE PONDS RAILWAY STATION,NaN,NaN,NaN,NaN,NaN,...,P,3039,NaN,1\PS808526,0,425555768,7,NaN,425944183,NaN
1101560,GAVIC424059888,2022-07-21,2022-08-09,NaN,ARARAT RAILWAY STATION WORKS DEPOT,NaN,NaN,NaN,NaN,NaN,...,P,3377,NaN,1\TP935273,1,424147731,7,NaN,128157181,NaN


In [66]:
GNAF_DFK['VIC_ADDRESS_DETAIL']['PRIMARY_SECONDARY'].value_counts()

PRIMARY_SECONDARY
S    1225385
P     165763
Name: count, dtype: int64

In [63]:
GNAF_DFK['VIC_STREET_LOCALITY_POINT'].query(f'STREET_LOCALITY_PID == "{street_locality_pid}"')

,STREET_LOCALITY_POINT_PID,DATE_CREATED,DATE_RETIRED,STREET_LOCALITY_PID,BOUNDARY_EXTENT,PLANIMETRIC_ACCURACY,LONGITUDE,LATITUDE
148734,L3149480,2021-11-12,NaN,VIC1972950,282,NaN,145.12385263,-37.90921935


In [75]:
GNAF_DFK['VIC_STREET_LOCALITY'].groupby('GNAF_STREET_PID')['STREET_NAME'].apply(list)

GNAF_STREET_PID
252834271      [A BLAIRS, A BLAIRS]
252834272                 [A BONES]
252834273    [A BROWNES, A BROWNES]
252834278              [A MCGARRYS]
252834281                [A SMITHS]
                      ...          
507741873         [MOUNT BROUGHTON]
507776268                 [MACLEAN]
507776269                  [MISERY]
507776270                 [SHILTON]
507781483                  [JOSEPH]
Name: STREET_NAME, Length: 80547, dtype: object

In [76]:
GNAF_DFK['VIC_STREET_LOCALITY']

,STREET_LOCALITY_PID,DATE_CREATED,DATE_RETIRED,STREET_CLASS_CODE,STREET_NAME,STREET_TYPE_CODE,STREET_SUFFIX_CODE,LOCALITY_PID,GNAF_STREET_PID,GNAF_STREET_CONFIDENCE,GNAF_RELIABILITY_CODE
0,VIC1962328,2021-07-06,NaN,C,CLARENCE,STREET,NaN,loc201e214973bd,253028435,2,4
1,VIC2031591,2021-07-06,NaN,C,STABLE,COURT,NaN,locb10305db77b5,253059724,2,4
2,VIC2049613,2021-07-06,NaN,C,TUDOR,COURT,NaN,loc00d1503504f1,253062827,2,4
3,VIC1967443,2021-07-06,NaN,C,DEIDRE,STREET,NaN,locd755ccb7197e,253031330,2,4
4,VIC1968231,2021-07-06,NaN,C,GEORGE,STREET,NaN,loca2dba6250aa2,253036166,2,4
...,...,...,...,...,...,...,...,...,...,...,...
202483,VIC2001775,2021-07-06,NaN,C,JANNALI,DRIVE,NaN,locd6810256cc93,253041297,2,4
202484,VIC3627604,2022-08-11,NaN,C,KAURI,WALK,NaN,loc90b2f4dd8c2d,505657921,2,4
202485,VIC2069872,2021-07-06,NaN,C,UNION,STREET,NaN,loc5d01e4efc615,253063178,2,4
202486,VIC2051726,2021-07-06,NaN,C,RUSSELL,STREET,NaN,loc20a81a4bf246,253057104,2,4


In [79]:
GNAF_DFK['Authority_Code_PS_JOIN_TYPE_AUT']

,CODE,NAME,DESCRIPTION
0,1,AUTO,"AUTOMATICALLY MATCHED PRIMARY AND SECONDARY, B..."
1,2,MANUAL,"MANUALLY GENERATED LINK, MAY OR MAY NOT HAVE T..."


In [64]:
GNAF_DFK['VIC_STREET_LOCALITY_POINT']['BOUNDARY_EXTENT'].unique()

array([nan, '33', '68', ..., '14498', '4586', '3854'], dtype=object)

In [68]:
GNAF_DFK['Authority_Code_']

,CODE,NAME,DESCRIPTION
0,1,PARCEL LEVEL MATCH,A parcel level geocode for the address has bee...
1,2,GAP GEOCODED ADDRESS LEVEL MATCH,A gap geocoded match for the address has been ...
2,3,STREET LOCALITY LEVEL SINGLE MATCH,A street-locality level geocode for the addres...
3,4,STREET LOCALITY LEVEL MULTIPLE MATCH,A street-locality level geocode for the addres...
4,5,LOCALITY LEVEL MULTIPLE MATCH,A locality level geocode for the address has b...


In [42]:
GNAF_DFK['VIC_STREET_LOCALITY']['STREET_NAME'].notna().all()
GNAF_DFK['VIC_STREET_LOCALITY'][GNAF_DFK['VIC_STREET_LOCALITY']['STREET_TYPE_CODE'].isna()]

,STREET_LOCALITY_PID,DATE_CREATED,DATE_RETIRED,STREET_CLASS_CODE,STREET_NAME,STREET_TYPE_CODE,STREET_SUFFIX_CODE,LOCALITY_PID,GNAF_STREET_PID,GNAF_STREET_CONFIDENCE,GNAF_RELIABILITY_CODE
451,VIC2070451,2021-07-06,NaN,C,THE MEWS,NaN,NaN,locd1d128ec42fa,253061684,2,4
576,VIC2027433,2021-07-06,NaN,C,THE AVENUE,NaN,NaN,loc4858bcc1d912,253061608,2,4
660,VIC2028892,2021-08-17,NaN,C,THE EYRIE,NaN,NaN,loc819aeb5db205,253061653,2,4
938,VIC2030389,2021-07-06,NaN,C,THE STRAND,NaN,NaN,loc9fc9cdac1e1a,253061730,2,4
939,VIC2030460,2021-07-06,NaN,C,THE WOODLAND,NaN,NaN,locb8f595af5fb8,253061741,2,4
...,...,...,...,...,...,...,...,...,...,...,...
202239,VIC2002940,2021-08-17,NaN,C,R O W Y,NaN,NaN,loc4fa4b090ce9e,NaN,NaN,4
202268,VIC2030928,2021-08-17,NaN,U,UNNAMED,NaN,NaN,loc8af2efa53ae5,NaN,NaN,4
202269,VIC2057561,2021-08-17,NaN,U,UNNAMED,NaN,NaN,loc154390ddc0ff,NaN,NaN,4
202274,VIC2057500,2021-08-17,NaN,U,UNNAMED,NaN,NaN,loc860b95b61d88,NaN,NaN,4


In [37]:
GNAF_DFK['VIC_ADDRESS_DETAIL'].query(f'STREET_LOCALITY_PID == "{street_locality_pid}" and NUMBER_FIRST == "39"').iloc[0]

ADDRESS_DETAIL_PID       GAVIC421529072
DATE_CREATED                 2004-04-29
DATE_LAST_MODIFIED           2021-07-07
DATE_RETIRED                        NaN
BUILDING_NAME                       NaN
LOT_NUMBER_PREFIX                   NaN
LOT_NUMBER                          NaN
LOT_NUMBER_SUFFIX                   NaN
FLAT_TYPE_CODE                      NaN
FLAT_NUMBER_PREFIX                  NaN
FLAT_NUMBER                         NaN
FLAT_NUMBER_SUFFIX                  NaN
LEVEL_TYPE_CODE                     NaN
LEVEL_NUMBER_PREFIX                 NaN
LEVEL_NUMBER                        NaN
LEVEL_NUMBER_SUFFIX                 NaN
NUMBER_FIRST_PREFIX                 NaN
NUMBER_FIRST                         39
NUMBER_FIRST_SUFFIX                 NaN
NUMBER_LAST_PREFIX                  NaN
NUMBER_LAST                         NaN
NUMBER_LAST_SUFFIX                  NaN
STREET_LOCALITY_PID          VIC1972950
LOCATION_DESCRIPTION                NaN
LOCALITY_PID            loc86dc9bf35404


In [33]:
GNAF_DFK['Authority_Code_GEOCODE_TYPE_AUT'].query('CODE == "FCS"').iloc[0]['DESCRIPTION']

'A POINT SET BACK FROM THE CENTRE OF THE ROAD FRONTAGE WITHIN AN ADDRESS SITE.'